# Introduction - Oracle Version 2

Notebook to load **bug reports**, **test cases** datasets and the **feature_matrixes** from the expert and volunteers responses given in the PyBossa applications, and create from them the **oracle** dataset. 

In this notebook we create a version of the oracle based on the results obtained from the empirical study made with volunteers and the PyBossa application. The relationship between bug reports and test cases is established through the firefox features shared between both artifacts.

Is expected that this oracle be more precise than the version created on the previous notebook (__oracle_v1__), once the trace links are created based on the existing relationship between the bug report and a given Firefox Feature.

## Load Libraries and Data

In [10]:
import pandas as pd
import numpy as np
from sklearn.externals.joblib import Parallel, delayed
from tqdm import tqdm

from utils import aux_functions

testcases = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/TC/testcases_final.csv')
print('Test Cases Shape: {}'.format(testcases.shape))

bugreports = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/bugreports_final.csv')
print('Bug Reports shape: {}'.format(bugreports.shape))

expert_matrix = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/br_feat_recovery_empirical_study/pybossa-apps/recover_taskruns/br_2_feature_matrix_expert.csv')
expert_matrix.set_index('bug_number', inplace=True)
print('Expert Matrix shape: {}'.format(expert_matrix.shape))

volunteers_matrix = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/br_feat_recovery_empirical_study/pybossa-apps/recover_taskruns/br_2_feature_matrix_volunteers.csv')
volunteers_matrix.set_index('bug_number', inplace=True)
print('Volunteers Matrix shape: {}'.format(volunteers_matrix.shape))

features = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/Features/features.csv')
print('Features shape: {}'.format(features.shape))

Test Cases Shape: (207, 12)
Bug Reports shape: (35336, 18)
Expert Matrix shape: (93, 21)
Volunteers Matrix shape: (30, 21)
Features shape: (21, 5)


## EDA - Exploratory Data Analysis

In [11]:
testcases.head()

,TC_Number,TestDay,Feature_ID,Firefox_Feature,Gen_Title,Crt_Nr,Title,Preconditions,Steps,Expected_Result,tc_name,tc_desc
0,1,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,1,Notification - Popup Block,NaN,1. Launch Firefox\n2. Navigate to http://www.p...,1. Firefox is successfully launched\n9. The al...,TC_1_TRG,1 20181221 20 <notificationbox> and <notificat...
1,2,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,2,Notification - Process Hang,NaN,"1. Launch Firefox\n2. In the URL bar, navigate...",1. Firefox is successfully launched\n2. Firefo...,TC_2_TRG,2 20181221 20 <notificationbox> and <notificat...
2,3,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,3,Verify Notifications appear in RTL Mode,NaN,"1. Launch Firefox\n2. In about:config, change ...",1. Firefox is successfully launched\n2.The for...,TC_3_TRG,3 20181221 20 <notificationbox> and <notificat...
3,4,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,4,Verify Notifications appear in High Contrast M...,NaN,"1. While the browser is in High Contrast Mode,...",1. Firefox has been launched.\n2. Firefox begi...,TC_4_TRG,4 20181221 20 <notificationbox> and <notificat...
4,5,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,5,Verify notifications react to differing Zoom l...,NaN,"1. While the browser is in High Contrast Mode,...",1. Firefox has been launched.\n2. Firefox begi...,TC_5_TRG,5 20181221 20 <notificationbox> and <notificat...


In [12]:
bugreports.head()

,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,Status,Product,Priority,Resolution,Severity,Is_Confirmed,br_name,br_desc
0,506297,Livemarks with null site/feed uris cause sync ...,All,Sync,unspecified,2009-07-24T17:08:43Z,NaN,NaN,2009-07-24 09:54:28 FaultTolerance D...,2009-07-24T17:08:43Z,RESOLVED,Firefox,--,FIXED,normal,True,BR_506297_SRC,506297 Livemarks with null site/feed uris caus...
1,506338,Enhance Crash Recovery to better help the user,All,Session Restore,Trunk,2009-07-24T19:17:21Z,[crashkill][crashkill-metrics],NaN,When our users crash they are pretty much in t...,2009-07-24T19:17:21Z,NEW,Firefox,--,NaN,enhancement,True,BR_506338_SRC,506338 Enhance Crash Recovery to better help t...
2,506507,Dragging multiple bookmarks in the bookmarks s...,x86,Bookmarks & History,Trunk,2009-07-26T06:16:02Z,NaN,NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T06:16:02Z,RESOLVED,Firefox,--,WORKSFORME,normal,True,BR_506507_SRC,506507 Dragging multiple bookmarks in the book...
3,506550,Unreliable Back Button navigating nytimes.com,x86,Extension Compatibility,3.5 Branch,2009-07-26T16:12:10Z,[caused by adblock plus][platform-rel-NYTimes],NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T16:12:10Z,RESOLVED,Firefox,--,FIXED,normal,False,BR_506550_SRC,506550 Unreliable Back Button navigating nytim...
4,506575,ALT + F4 when dropdown of autocomplete is open...,x86,Address Bar,3.5 Branch,2009-07-26T20:14:54Z,NaN,NaN,Pressing ALT + F4 when the autocomplete dropdo...,2009-07-26T20:14:54Z,NEW,Firefox,P5,NaN,normal,True,BR_506575_SRC,506575 ALT + F4 when dropdown of autocomplete ...


In [13]:
features.head()

,Feature_Number,Firefox_Version,Firefox_Feature,Feature_Description,Reference
0,1,48 Branch + 50 Branch,New Awesome Bar,The Firefox address bar displays a page's web ...,https://support.mozilla.org/en-US/kb/awesome-b...
1,2,48 Branch,Windows Child Mode,Child mode is a feature of Windows that allows...,https://wiki.mozilla.org/QA/Windows_Child_Mode
2,3,48 Branch,APZ - Async Scrolling,The Async Pan/Zoom module (APZ) is a platform ...,https://wiki.mozilla.org/Platform/GFX/APZ
3,4,49 Branch,Browser Customization,. Install and Customize Firefox Themes. \n. Cu...,https://support.mozilla.org/en-US/kb/use-theme...
4,5,49 Branch,PDF Viewer,"Zoom in, Zoom out, Print and Save PDF Files.",https://support.mozilla.org/en-US/kb/view-pdf-...


In [14]:
expert_matrix.head()

,new_awesome_bar,windows_child_mode,apz_async_scrolling,browser_customization,pdf_viewer,context_menu,w10_comp,tts_in_desktop,tts_in_rm,webgl_comp,...,pointer_lock_api,webm_eme,zoom_indicator,downloads_dropmaker,webgl2,flac_support,indicator_device_perm,flash_support,notificationbox,update_directory
bug_number,,,,,,,,,,,,,,,,,,,,,
1181835,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1248267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1248268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1257087,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1264988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
volunteers_matrix.head()

,new_awesome_bar,windows_child_mode,apz_async_scrolling,browser_customization,pdf_viewer,context_menu,w10_comp,tts_in_desktop,tts_in_rm,webgl_comp,...,pointer_lock_api,webm_eme,zoom_indicator,downloads_dropmaker,webgl2,flac_support,indicator_device_perm,flash_support,notificationbox,update_directory
bug_number,,,,,,,,,,,,,,,,,,,,,
1181835,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1248267,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1248268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1257087,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1264988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Bug Reports Selection

In [16]:
brs_versions = ['48 Branch', '49 Branch', '50 Branch', '51 Branch']
brs_status = ['RESOLVED','VERIFIED']
brs_priority = ['P1', 'P2', 'P3']
brs_resolutions = ['FIXED']
brs_severities = ['major', 'normal', 'blocker', 'critical']
brs_isconfirmed = [True]
selected_bugs = bugreports[(bugreports.Version.isin(brs_versions)) &
                             (bugreports.Status.isin(brs_status)) &
                             (bugreports.Priority.isin(brs_priority)) &
                             (bugreports.Resolution.isin(brs_resolutions)) &
                             (bugreports.Severity.isin(brs_severities)) &
                             (bugreports.Is_Confirmed.isin(brs_isconfirmed))
                            ]
print(selected_bugs.shape)

(93, 18)


## Oracle

#### Add Feature_Name column to features dataset

In [17]:
features['Feature_Name'] = ['new_awesome_bar',
            'windows_child_mode',
            'apz_async_scrolling',
            'browser_customization',
            'pdf_viewer',
            'context_menu',
            'w10_comp', 
            'tts_in_desktop', 
            'tts_in_rm', 
            'webgl_comp',
            'video_and_canvas_render', 
            'pointer_lock_api',
            'webm_eme', 
            'zoom_indicator',
            'downloads_dropmaker',
            'webgl2', 
            'flac_support', 
            'indicator_device_perm',
            'flash_support',  
            'notificationbox',          
            'update_directory']
features.head(22)

,Feature_Number,Firefox_Version,Firefox_Feature,Feature_Description,Reference,Feature_Name
0,1,48 Branch + 50 Branch,New Awesome Bar,The Firefox address bar displays a page's web ...,https://support.mozilla.org/en-US/kb/awesome-b...,new_awesome_bar
1,2,48 Branch,Windows Child Mode,Child mode is a feature of Windows that allows...,https://wiki.mozilla.org/QA/Windows_Child_Mode,windows_child_mode
2,3,48 Branch,APZ - Async Scrolling,The Async Pan/Zoom module (APZ) is a platform ...,https://wiki.mozilla.org/Platform/GFX/APZ,apz_async_scrolling
3,4,49 Branch,Browser Customization,. Install and Customize Firefox Themes. \n. Cu...,https://support.mozilla.org/en-US/kb/use-theme...,browser_customization
4,5,49 Branch,PDF Viewer,"Zoom in, Zoom out, Print and Save PDF Files.",https://support.mozilla.org/en-US/kb/view-pdf-...,pdf_viewer
5,6,49 Branch,Context Menu,The Context Menu is the menu that appears when...,https://www.paessler.com/manuals/paessler_site...,context_menu
6,7,49 Branch,Windows 10 Compatibility,". Users can browse through different sites, \n...",https://support.mozilla.org/en-US/kb/how-downl...,w10_comp
7,8,49 Branch,Text to Speech on Desktop,Extension of Firefox to read out loud the HTML...,https://addons.mozilla.org/en-US/firefox/addon...,tts_in_desktop
8,9,49 Branch,Text to Speech in Reader Mode,Extension of Firefox to read out loud the cont...,https://support.mozilla.org/en-US/kb/firefox-r...,tts_in_rm
9,10,49 Branch,WebGL Compatibility,Feature of Firefox implemted based on WebGL li...,https://pt.wikipedia.org/wiki/WebGL,webgl_comp


In [18]:
display(expert_matrix.head())
expert_matrix.iloc[0,5]

,new_awesome_bar,windows_child_mode,apz_async_scrolling,browser_customization,pdf_viewer,context_menu,w10_comp,tts_in_desktop,tts_in_rm,webgl_comp,...,pointer_lock_api,webm_eme,zoom_indicator,downloads_dropmaker,webgl2,flac_support,indicator_device_perm,flash_support,notificationbox,update_directory
bug_number,,,,,,,,,,,,,,,,,,,,,
1181835,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1248267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1248268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1257087,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1264988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


1

In [19]:
def get_features_from_expert_matrix(br_id):
    features_ids = ""
    for col in expert_matrix.columns:
        if expert_matrix.at[br_id, col] == 1:
            if features_ids == "":
                features_ids = str(expert_matrix.columns.get_loc(col) + 1)
            else:
                features_ids = features_ids + " " + str(expert_matrix.columns.get_loc(col) + 1)
    return features_ids

selected_bugs['Features_IDs'] = ""
for br_id in expert_matrix.index:
    for idx2, br in selected_bugs.iterrows():
        if br.Bug_Number == br_id:
            selected_bugs.at[idx2, 'Features_IDs'] = get_features_from_expert_matrix(br_id)

selected_bugs[['Bug_Number','Features_IDs']].head()

/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Bug_Number,Features_IDs
9192,1181835,6
10621,1248267,
10622,1248268,
10904,1257087,1
11150,1264988,


In [20]:
selected_bugs.head()

,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,Status,Product,Priority,Resolution,Severity,Is_Confirmed,br_name,br_desc,Features_IDs
9192,1181835,Provide a UI for migrating users' add-ons to w...,Unspecified,Extension Compatibility,49 Branch,2015-07-08T23:01:45Z,[UX] triaged,NaN,We're still not exactly sure how this would wo...,2015-07-08T23:01:45Z,RESOLVED,Firefox,P2,FIXED,normal,True,BR_1181835_SRC,1181835 Provide a UI for migrating users' add-...,6
10621,1248267,"Right click on bookmark item of ""Recently Book...",Unspecified,Bookmarks & History,48 Branch,2016-02-14T17:45:54Z,NaN,NaN,Steps To Reproduce: 1. Open Bookmarks menu 2. ...,2016-02-14T17:45:54Z,VERIFIED,Firefox,P3,FIXED,normal,True,BR_1248267_SRC,"1248267 Right click on bookmark item of ""Recen...",
10622,1248268,"Unable to disable ""Recently bookmarked""",All,Bookmarks & History,48 Branch,2016-02-14T17:54:44Z,NaN,NaN,Created attachment 8719295 Firefox Nightly 47_...,2016-02-14T17:54:44Z,VERIFIED,Firefox,P3,FIXED,major,True,BR_1248268_SRC,"1248268 Unable to disable ""Recently bookmarked...",
10904,1257087,Middle mouse click on history item would not open,Unspecified,Bookmarks & History,48 Branch,2016-03-16T05:13:47Z,NaN,NaN,[Tracking Requested - why for this release]: r...,2016-03-16T05:13:47Z,VERIFIED,Firefox,P2,FIXED,normal,True,BR_1257087_SRC,1257087 Middle mouse click on history item wou...,1
11150,1264988,Scrollbar appears for a moment in the new Awes...,All,Address Bar,48 Branch,2016-04-15T15:17:33Z,[fxsearch] [photon-performance],NaN,Created attachment 8741829 Bug.mov User Agent...,2016-04-15T15:17:33Z,VERIFIED,Firefox,P1,FIXED,normal,True,BR_1264988_SRC,1264988 Scrollbar appears for a moment in the ...,


In [21]:
def check_link_condition(br, tc):
    if str(tc.Feature_ID) in br.Features_IDs.split(" "):
            return True
    return False

oracle_df = pd.DataFrame(columns=selected_bugs.br_name, index=testcases.tc_name, data=np.zeros(shape=(len(testcases),len(selected_bugs))), dtype='int8')
for idx_1,br in tqdm(selected_bugs.iterrows()):
    for idx_2,tc in testcases.iterrows():
        if check_link_condition(br, tc):
            oracle_df.at[tc.tc_name, br.br_name] = 1
        else:
            oracle_df.at[tc.tc_name, br.br_name] = 0


93it [00:01, 47.53it/s]


In [25]:
print('Oracle shape: {}\n'.format(oracle_df.shape))
display(oracle_df.head())

Oracle shape: (207, 93)



br_name,BR_1181835_SRC,BR_1248267_SRC,BR_1248268_SRC,BR_1257087_SRC,BR_1264988_SRC,BR_1267480_SRC,BR_1267501_SRC,BR_1269348_SRC,BR_1269485_SRC,BR_1270274_SRC,...,BR_1352539_SRC,BR_1353831_SRC,BR_1357085_SRC,BR_1357458_SRC,BR_1365887_SRC,BR_1408361_SRC,BR_1430603_SRC,BR_1432915_SRC,BR_1449700_SRC,BR_1451475_SRC
tc_name,,,,,,,,,,,,,,,,,,,,,
TC_1_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TC_2_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TC_3_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TC_4_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TC_5_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Save Datasets
New Oracle and Selected Bug Reports Dataset
Dimensions: 
 * 207 x 93
 * 93 x 19

In [84]:
oracle_df.to_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final_emp_study.csv')
selected_bugs.to_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/selected_bugreports_final.csv', index=False)

## -----------------------------------------

#### Checking Values [0]

#### Analyze Entire Oracle Created

In [85]:
oo_df_full = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final_emp_study.csv')
oo_df_full.set_index('tc_name', inplace=True)

assert(oo_df_full.loc['TC_13_TRG', 'BR_1257087_SRC'] == 1)
assert(oo_df_full.loc['TC_14_TRG', 'BR_1271607_SRC'] == 1)
assert(oo_df_full.loc['TC_15_TRG', 'BR_1276120_SRC'] == 1)

### -----

#### Checking Values [1]
Amount Positive and Negative Links

In [86]:
positive_links = 0
negative_links = 0
for idx,row in oo_df_full.iterrows():
    for col in oo_df_full.columns:
        if row[col] == 1:
            positive_links = positive_links + 1
        else:
            negative_links = negative_links + 1

print("Positive Links Amount: {}".format(positive_links))
print("Negative Links Amount: {}".format(negative_links))

Positive Links Amount: 915
Negative Links Amount: 18336


##### Expected Amount of Positive and Negative Links

In [87]:
for idx, row in selected_bugs.iterrows():
    amount_tcs = 0  # amount of testcases of feature
    for f_id in row.Features_IDs.split(" "):
        if f_id != "":
            amount_tcs = amount_tcs + len(testcases[testcases.Feature_ID == int(f_id)])

    selected_bugs.at[idx, 'Amount_TCs'] = amount_tcs
    
display(selected_bugs[['Bug_Number', 'Features_IDs', 'Amount_TCs']].head(10))

positives_amount = selected_bugs.Amount_TCs.sum()
negatives_amount = len(selected_bugs) * len(testcases) - positives_amount

print("Total Amount of Expected Positive Links: {}".format(positives_amount))
print("Total Amount of Expected Negative Links: {}".format(negatives_amount))

/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/guilherme/anaconda3/envs/trace-link-recovery-study/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Bug_Number,Features_IDs,Amount_TCs
9192,1181835,6,31.0
10621,1248267,,0.0
10622,1248268,,0.0
10904,1257087,1,13.0
11150,1264988,,0.0
11229,1267480,3,22.0
11232,1267501,,0.0
11331,1269348,3,22.0
11339,1269485,,0.0
11375,1270274,6,31.0


Total Amount of Expected Positive Links: 915.0
Total Amount of Expected Negative Links: 18336.0


### ---------